In [19]:
# Install OpenCV
!pip install opencv-python

# Install NumPy
!pip install numpy

# Install TensorFlow (including TensorFlow Hub)
!pip install tensorflow tensorflow-hub

# Install Matplotlib (for displaying images in Jupyter Notebook)
!pip install matplotlib


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [7]:
import os

video_path = r'C:\Users\dell\Downloads\SampleVideo.mp4'  # Use raw string or double backslashes
if not os.path.isfile(video_path):
    print("File does not exist.")
else:
    print("File found.")


File found.


In [2]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
from IPython.display import display, clear_output



In [3]:
# Load pre-trained model for object detection from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1")



In [4]:
def detect_objects(frame):
    if frame is None:
        print("Warning: Empty frame encountered.")
        return frame

    # Resize frame for faster processing
    frame_resized = cv2.resize(frame, (640, 480))
    rgb_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(rgb_frame, dtype=tf.uint8)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Perform object detection
    detections = model(input_tensor)
    detection_boxes = detections['detection_boxes'].numpy()[0]
    detection_classes = detections['detection_classes'].numpy()[0].astype(np.int32)
    detection_scores = detections['detection_scores'].numpy()[0]

    # Draw bounding boxes and labels
    h, w, _ = frame_resized.shape
    for i in range(detection_boxes.shape[0]):
        score = detection_scores[i]
        if score > 0.5:  # Confidence threshold
            box = detection_boxes[i]
            class_id = detection_classes[i]

            ymin, xmin, ymax, xmax = box
            start_point = (int(xmin * w), int(ymin * h))
            end_point = (int(xmax * w), int(ymax * h))

            # Draw bounding box
            cv2.rectangle(frame_resized, start_point, end_point, (255, 0, 0), 2)

            # Draw label
            label = f"Class {class_id} Score: {score:.2f}"
            cv2.putText(frame_resized, label, (start_point[0], start_point[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame_resized

def process_video(video_path):
    # Check if video file exists
    if not os.path.isfile(video_path):
        print("Error: Unable to open video file. File does not exist.")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    frame_skip = 5  # Process every 5th frame
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        if frame_count % frame_skip == 0:
            # Apply object detection
            frame = detect_objects(frame)

            # Display the frame
            cv2.imshow('Video', frame)

            # Exit on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        frame_count += 1
    cap.release()
    cv2.destroyAllWindows()
    print("Processing complete.")

# Example usage
video_path = r'C:\Users\dell\Downloads\SampleVideo.mp4'  # Replace with your video file path
process_video(video_path)

End of video or error reading frame.
Processing complete.


In [5]:
import cv2
import numpy as np
import os

def detect_objects(frame):
    if frame is None:
        print("Warning: Empty frame encountered.")
        return frame

    # Resize frame for faster processing
    frame_resized = cv2.resize(frame, (640, 480))
    rgb_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(rgb_frame, dtype=tf.uint8)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Perform object detection (assuming `model` is predefined)
    detections = model(input_tensor)
    detection_boxes = detections['detection_boxes'].numpy()[0]
    detection_classes = detections['detection_classes'].numpy()[0].astype(np.int32)
    detection_scores = detections['detection_scores'].numpy()[0]

    # Draw bounding boxes and labels
    h, w, _ = frame_resized.shape
    for i in range(detection_boxes.shape[0]):
        score = detection_scores[i]
        if score > 0.5:  # Confidence threshold
            box = detection_boxes[i]
            class_id = detection_classes[i]

            ymin, xmin, ymax, xmax = box
            start_point = (int(xmin * w), int(ymin * h))
            end_point = (int(xmax * w), int(ymax * h))

            # Draw bounding box
            cv2.rectangle(frame_resized, start_point, end_point, (255, 0, 0), 2)

            # Draw label
            label = f"Class {class_id} Score: {score:.2f}"
            cv2.putText(frame_resized, label, (start_point[0], start_point[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame_resized

def track_motion(prev_frame_gray, curr_frame_gray):
    # Calculate the dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, curr_frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Normalize the magnitude
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Convert the angle to hue (HSV color representation)
    hsv = np.zeros_like(cv2.cvtColor(curr_frame_gray, cv2.COLOR_GRAY2BGR))
    hsv[..., 1] = 255  # Full saturation
    hsv[..., 0] = angle * 180 / np.pi / 2  # Angle in degrees
    hsv[..., 2] = magnitude  # Magnitude as brightness

    # Convert HSV to RGB for display
    motion_frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return motion_frame

def process_video_with_motion_tracking(video_path):
    # Check if video file exists
    if not os.path.isfile(video_path):
        print("Error: Unable to open video file. File does not exist.")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    frame_skip = 5  # Process every 5th frame
    frame_count = 0

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Unable to read the first frame.")
        cap.release()
        return

    # Resize the previous frame for consistent size
    prev_frame = cv2.resize(prev_frame, (640, 480))

    # Convert the first frame to grayscale
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break

        if frame_count % frame_skip == 0:
            # Resize current frame to match previous frame
            frame = cv2.resize(frame, (640, 480))

            # Convert current frame to grayscale
            curr_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Apply object detection
            detected_frame = detect_objects(frame)

            # Apply motion tracking
            motion_frame = track_motion(prev_frame_gray, curr_frame_gray)

            # Resize motion_frame to match detected_frame if dimensions differ
            motion_frame = cv2.resize(motion_frame, (detected_frame.shape[1], detected_frame.shape[0]))

            # Display the object detection frame and motion tracking frame side by side
            combined_frame = np.hstack((detected_frame, motion_frame))
            cv2.imshow('Object Detection & Motion Tracking', combined_frame)

            # Update previous frame for motion tracking
            prev_frame_gray = curr_frame_gray

            # Exit on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
    print("Processing complete.")

# Example usage
video_path = r'C:\Users\dell\Downloads\SampleVideo.mp4'  # Replace with your video file path
process_video_with_motion_tracking(video_path)


End of video or error reading frame.
Processing complete.
